# VISUALISATION USING DASH (go)

In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql
import pickle

# Plotly
import plotly.graph_objects as go

# Web app
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

database_name = "data/data.db"

In [2]:
conn = sql.connect(database_name)
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM clean_data", conn)
conn.close()

df.head(1)

,YQuarter,Ticker,gsector,Date,X1_REVGH,X2_EPS,X3_ROA,X4_ROE,X5_PE,X6_PS,...,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12
0,1999Q3,A,Sector_35,1999-07-31 00:00:00,0.000267,0.251872,0.791643,0.974791,0.515213,0.000012,...,-0.08238,0.037696,-0.001543,-0.001425,-0.000367,0.0001,0.000303,0.000051,-0.000362,-0.000199


In [ ]:
# conn = sql.connect("data/data.db")
# cur = conn.cursor()
# # PCA
# np.exp(df.select_dtypes(include=['float64']).iloc[:,-12:]).to_sql("pca",conn,index=False,if_exists='replace')
# # Features
# df.select_dtypes(include=['float64']).iloc[:, :20].to_sql("features",conn,index=False,if_exists='replace')
# # Clusters
# df[['Cluster']].to_sql("clusters",conn,index=False,if_exists='replace')
# # Index 
# df.iloc[:,:4].to_sql("other",conn,index=False,if_exists='replace')
# conn.close()

## DASH APP 3 
- build app 2 plots with plotly go and callbacks
- added customization (Because needs an input, but selections dont work)

- [Great 3D scatter examples (go)](https://plot.ly/python/3d-scatter-plots/)
- [Soft cluster clouds](https://plot.ly/python/v3/3d-point-clustering/)
- [Dash app gallery](https://dash-gallery.plotly.host/Portal/))
- [My Example](https://github.com/09acp/Stock-Dashboard-Momentum-Factor-/blob/master/app.py)

In [ ]:
#df.head(1)
# df.Cluster.value_counts()
# k = np.exp(df.select_dtypes(include=['float64']).iloc[:,-12:])
# k.head()

In [ ]:
# DEFINE HTML TEMPLATE
app = dash.Dash() # Boostrap CSS.
app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'}) 

app.layout = html.Div([
        
    dcc.Tabs(id="tabs", children=[  # ALL TABS START

        # TAB 1. 3D Scatter Plot: PCA & Clustering
        dcc.Tab(label= '3D PCA & Clustering', children=[
            html.Div([
                html.H1(" Three Component Scatter Plot ", style={'textAlign': 'center'}),
                
                #1
                dcc.DatePickerRange(id = "date_1",
                                    min_date_allowed=df['Date'].min(),
                                    max_date_allowed=df['Date'].max(),
                                    start_date=df['Date'].min(),
                                    end_date=df['Date'].max()      ),
                #2
                html.Button(id='ticker_1', 
                            n_clicks = 0,
                            children = "Submit"),
                #3
                dcc.Graph(id = "scatter_1"),
                        ]),
        ]),  # END TAB 1

   ]), # ALL TABS END  
], className="container")

@app.callback(Output('scatter_1', 'figure'),            
              [Input('ticker_1', 'n_clicks')], #ACTIVATION 2 
              [State('date_1', 'start_date'), 
               State('date_1', 'end_date')])

def scatter_3D ( n_clicks, start_date, end_date ):
    trace0 = go.Scatter3d( 
                         x = df["PCA_1"],   # Exponential PCA
                         y = df["PCA_2"],
                         z = df["PCA_3"],  #)
                         mode = "markers",
                         marker = dict(size= (np.exp(df["y_return"])*10),   # SCALE FIX
                                        color=df["Cluster"],
                                        colorscale = "Portland",
                                        opacity = 0.3, 
                                        showscale = False,   )  )
    data = [ trace0 ]
    layout = {"title":"3D PCA Scatter Plot",
              "xaxis": {"title":"PCA 1"},
              "yaxis": {"title":"PCA 2"},
              "zaxis": {"title":"PCA 3"},}
             
    fig = {"data":data,
           "layout":layout}
    
    #return tools.create_plot(fig)
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    return fig



In [ ]:
if __name__ == '__main__':
    app.run_server()

## DASH APP 4 
User Customization features 
- [Plotly components](https://plot.ly/python/reference/#mesh3d)
- [ ] Soft clusters. [Examples](https://plot.ly/python/3d-mesh/)
- [ ] Years slider
- [ ] Quartile Dropdown

##### Frequent queries make it crash

In [ ]:

" MESH SUBPLOT DOESNT WORK"
#     trace0 = go.Mesh3d(x = df["PCA_1"],   # Exponential PCA
#                        y = df["PCA_2"],
#                        z = df["PCA_3"],  #)
#                        alphahull=7,
#                        opacity=0.1,
#                        color='cyan')
    
#     trace1 = go.Scatter3d(x = df["PCA_1"],   # Exponential PCA
#                           y = df["PCA_2"],
#                           z = df["PCA_3"],  #)
#                           mode = "markers",
#                           marker = dict(size= (np.exp(df["y_return"])*10),   # SCALE FIX
#                                         color=df["y_return"],  # THIS CAN BE DIFFERENT
#                                         colorscale = "Portland",
#                                         showscale = False,   )  )

In [ ]:
quarter_options = [{'label': i, 'value': i} for i in list(df["YQuarter"].unique())]

# DEFINE HTML TEMPLATE
app = dash.Dash() # Boostrap CSS.
app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'}) 

app.layout = html.Div([
        
    dcc.Tabs(id="tabs", children=[  # ALL TABS START

        # TAB 1. 3D Scatter Plot: PCA & Clustering
        dcc.Tab(label= '3D PCA & Clustering', children=[
            html.Div([
                html.H1(" Three Component Scatter Plot ", style={'textAlign': 'center'}),
                
                #1
                dcc.DatePickerRange(id = "date_1",
                                    min_date_allowed=df['Date'].min(),
                                    max_date_allowed=df['Date'].max(),
                                    start_date=df['Date'].min(),
                                    end_date=df['Date'].max()      ),
                #2
                html.Button(id='ticker_1', 
                            n_clicks = 0,
                            children = "Submit"),
                #3
                dcc.Dropdown(id='quarter_1',
                             options= quarter_options, # df["YQuarter"].tolist(), #df["YQuarter"],
                             multi=True,
                             value=None,style={"display": "block","margin-left": "auto","margin-right": "auto","width": "60%"}
                             ),                
                #4
                dcc.Graph(id = "scatter_1"),
                        ]),
        ]),  # END TAB 1

   ]), # ALL TABS END  
], className="container")

In [ ]:
@app.callback(Output('scatter_1', 'figure'),            
              [Input('ticker_1', 'n_clicks'),
               Input('quarter_1','value'),    ], #ACTIVATION 2 
              [State('date_1', 'start_date'), 
               State('date_1', 'end_date')])

def scatter_3D ( n_clicks, quarter_range, start_date, end_date ):  # FIX
    # Call data from SQL db
    
    
    conn = sql.connect(database_name)
    cur = conn.cursor()
    #df = pd.read_sql_query("SELECT * FROM clean_data", conn)
    data = pd.read_sql_query("SELECT * FROM clean_data WHERE Date >= '{}' AND Date <= '{}'"
                           .format(start_date,end_date), conn)
    conn.close()    
    
    trace1 = go.Scatter3d(x = data["PCA_1"],   # Exponential PCA
                          y = data["PCA_2"],
                          z = data["PCA_3"],  #)
                          mode = "markers+text",
                          marker = dict(size= np.exp(data["y_return"])*10,   # SCALE FIX
                                        color= data["y_return"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        showscale = False,   )  )

    conn.close() # Close DB
    
    data = [ trace1 ]
    layout = {"title":"3D PCA Scatter Plot",
              "xaxis": {"title":"PCA 1"},
              "yaxis": {"title":"PCA 2"},
              "zaxis": {"title":"PCA 3"},}
             
    fig = {"data":data,
           "layout":layout}
    
    return fig

if __name__ == '__main__':
    app.run_server()

## DASH APP 5 
***FOR DATE RANGE TO WORK IT MUST BE IN THE CORRECT FORMAT -DatetimeIndex etc***

User Customization features 
- [x] Date RangeSlider
- [Dash Sliders](https://dash.plot.ly/dash-core-components/rangeslider)
- [Date Slider Forum](https://community.plot.ly/t/solved-has-anyone-made-a-date-range-slider/6531/2)
- **[Dash Slider app](https://medium.com/@divyakarthik28/moving-average-crossover-for-stocks-using-dash-d6fc91702999)**

In [ ]:
quarter_options = [{'label': i, 'value': i} for i in list(df["YQuarter"].unique())]
#date_options = [{'label': i[:10], 'value': i} for i in df["Date"].unique()] # [{'label': i, 'value': i} for i in list(df["Date"].unique())]
year_options = pd.DatetimeIndex(df['Date']).year

# DEFINE HTML TEMPLATE
app = dash.Dash() # Boostrap CSS.
app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'}) 

app.layout = html.Div([
        
    dcc.Tabs(id="tabs", children=[  # ALL TABS START

        # TAB 1. 3D Scatter Plot: PCA & Clustering
        dcc.Tab(label= '3D PCA & Clustering', children=[
            html.Div([
                html.H1(" Three Component Scatter Plot ", style={'textAlign': 'center'}),
                
                #1              
                # https://community.plot.ly/t/solved-has-anyone-made-a-date-range-slider/6531
                dcc.RangeSlider( id = "date_1",
                                 updatemode = "drag",  #'drag' #don't let it update till mouse released
                                 min = year_options.min(),
                                 max = year_options.max(),
                                 marks = {i: '{}'.format(i) for i in year_options.unique().tolist()},
                                 value=[2013, 2014],                                                    ),
                                 # className="row",  
                                 # style={"padding-bottom": 30,"width":"80%","margin":"auto"}      ),
                #2
                html.Div(id='slider_selection'),
                #3
                html.Button(id='ticker_1', 
                            n_clicks = 0,
                            children = "Submit"),
                #4
                dcc.Dropdown(id='quarter_1',
                             options= quarter_options, 
                             multi=True,
                             value= ['2013Q1', '2014Q1'] ,
                             style={"display": "block","margin-left": "auto","margin-right": "auto","width": "40%"}
                             ),                
                #5
                dcc.Graph(id = "scatter_1"),
                        ]),
        ]),  # END TAB 1

   ]), # ALL TABS END  
], className="container")

In [ ]:
@app.callback([Output('scatter_1', 'figure'),
               Output('slider_selection', 'children')],
              [Input ('date_1','value'),
               Input('ticker_1', 'n_clicks'),
               Input('year_1','value'),           ]   ) 

def scatter_3D (rangeD, n_clicks, quarter_range  ):  # FIX
    # Call data from SQL db
    " NEED TO HAVE ALL INFO "
    conn = sql.connect(database_name)
    cur = conn.cursor()
    df2 = pd.read_sql_query("SELECT * FROM clean_data", conn)
    df2["Date"] = pd.DatetimeIndex(df2["Date"]).year
#     data = pd.read_sql_query("SELECT * FROM clean_data WHERE Date >= '{}' AND Date <= '{}'"
#                            .format(start_date,end_date), conn)
    conn.close()    
    
    dataset = df2[(df2["Date"] >= rangeD[0]) & (df2["Date"] <= rangeD[1])]
    trace1 = go.Scatter3d(x = dataset["PCA_1"],   # Exponential PCA
                          y = dataset["PCA_2"],
                          z = dataset["PCA_3"],  #)
                          mode = "markers",
                          marker = dict(size= np.exp(dataset["y_return"])*10,   # SCALE FIX
                                        color= dataset["y_return"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        showscale = False,   )  )

    conn.close() # Close DB
    
    data = [ trace1 ]
    layout = {"title":"3D PCA Scatter Plot",
              "xaxis": {"title":"PCA 1"},
              "yaxis": {"title":"PCA 2"},
              "zaxis": {"title":"PCA 3"},}
    
#     layout = {"title":"3D PCA Scatter Plot",
#               "xaxis": {
#               "rangeselector": {"title":"PCA 2",
#                                 "buttons": [  {"count": 6, "label": "6M", "step": "month",
#                                               "stepmode": "backward"},
#                                               {"count": 1, "label": "1Y", "step": "year",
#                                               "stepmode": "backward"},
#                                               {"count": 1, "label": "YTD", "step": "year",
#                                               "stepmode": "todate"},
#                                               {"label": "5Y", "step": "all",
#                                               "stepmode": "backward"}]                   } },
#               "yaxis": {"title":"PCA 2"},
#               "zaxis": {"title":"PCA 3"},}
    
    fig = {"data":data,
           "layout":layout}
    
    return fig , "Date Range {} - {}.".format(str(rangeD[0]),str(rangeD[1]))

if __name__ == '__main__':
    app.run_server()

===============================================

## DASH APP 6 
- [x] Quarter Dropdown 
- [x] Sector CheckList
    - ~~[ ] Need Dynamic Dropdown~~ [1](https://community.plot.ly/t/updating-a-dropdown-menus-contents-dynamically/4920/2); [2](https://dash.plot.ly/getting-started-part-2). Otherwise options outside date range are still available.
    - Cannot be made. Requires dictionary of all values between ranges
- [x] Symbol markers for each Sector
    - ~~I think it has to be an integer~~, [Solution1 ?](https://community.plot.ly/t/getting-different-markers-shapes/9944), [Solution2 ?](https://community.plot.ly/t/still-open-getting-different-marker-shapes-in-a-dash-app/11928/6)
- [x] Fix Layout 
- [x] Fix Drag and Axis
- [ ] Year Tabs (1yrs, 2yrs, 3yrs, all)
- [ ] [Feature Info and Ticker](https://plot.ly/python/text-and-annotations/)
- [ ] [Feature Info and Ticker GOOD](https://plot.ly/python/hover-text-and-formatting/)
- Returns replace PCA_3 in z-axis

In [258]:
quarter_options = [{'label': i, 'value': i} for i in list(df["YQuarter"].unique())]
year_options = pd.DatetimeIndex(df['Date']).year
# sector_options = []   # ALL SELECTED
# sector_options.append({'label': 'All', 'value': df["gsector"].unique().tolist()})
sector_options = [{'label': i, 'value': i} for i in list(pd.Series(df["gsector"].unique()).sort_values(axis=0, ascending=True))]

# DEFINE HTML TEMPLATE
app = dash.Dash() # Boostrap CSS.
app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'}) 

app.layout = html.Div([
    dcc.Tabs(id="tabs", children=[  # ALL TABS START
        
        # TAB 1. 3D Scatter Plot: PCA & Clustering
        dcc.Tab(label= '3D PCA & Clustering', children=[
            html.Div([
                html.H1(" Three Component Scatter Plot ", style={'textAlign': 'center'}),
                #1              
                # https://community.plot.ly/t/solved-has-anyone-made-a-date-range-slider/6531
                dcc.RangeSlider( id = "date_1",
                                 updatemode = "drag",  #'drag' #don't let it update till mouse released
                                 min = year_options.min(),
                                 max = year_options.max(),
                                 marks = {i: '{}'.format(i) for i in year_options.unique().tolist()},
                                 value=[2013, 2015],                                                    ),
                #2
                html.Div(id='slider_selection'),
                #3
                html.Button(id='ticker_1', 
                            n_clicks = 0,
                            children = "Submit"),
                #4
                dcc.Dropdown(id='quarter_1',
                             options=quarter_options, 
                             multi=True,
                             value= ['2013Q1'] ,  #, '2014Q1'
                             style={"display": "block","margin-left": "auto","margin-right": "auto","width": "40%"}
                             ),                
                #5
                dcc.Checklist(id='sector_1',
                              options=sector_options,
                              value= ['Sector_35', 'Sector_15', 'Sector_45', 'Sector_20', 
                                      'Sector_40', 'Sector_25', 'Sector_30', 'Sector_55', 
                                      'Sector_50', 'Sector_60', 'Sector_10'] ,             ),
                
                #6
                dcc.Graph(id = "scatter_1"),
                        ]),
            
        ]),  # END TAB 1
        
   ]), # ALL TABS END  
], className="container")

In [259]:
@app.callback([Output('scatter_1', 'figure'),
               Output('slider_selection', 'children')],
              [Input ('date_1','value'),
               Input('ticker_1', 'n_clicks'),
               Input('quarter_1','value'),          
               Input('sector_1','value'),           ]   ) 

def scatter_3D (rangeD, n_clicks, quarterD, sectorD ):  # FIX
    # Call data from SQL db
    " NEED TO HAVE ALL INFO "
    conn = sql.connect(database_name)
    cur = conn.cursor()
    df2 = pd.read_sql_query("SELECT * FROM clean_data", conn)
    df2["Date"] = pd.DatetimeIndex(df2["Date"]).year
    conn.close()    
    
    dataset = df2[(df2["Date"] >= rangeD[0]) &  # Slice data according to lower year range 
                   (df2["Date"] <= rangeD[1]) & # Slice data according to upper year range
                   (df2["YQuarter"].isin(quarterD)) & # Slice data according to quarter                       
                   (df2["gsector"].isin(sectorD))  # Slice data according to sector                            
                  ]  
    #dataset = dataset0[ dataset0['YQuarter'].isin(quarterD)] # Slice data according to quarter dropdown
    
    
    # Make sector into int for Symbols 
#     sector_encoding = dict(enumerate(dataset['gsector'].unique().tolist())) # crete numbered dict
#     sector_encoding2 = {v: k for k, v in sector_encoding.items()}  #inverse k-v
#     dataset['gsector'].replace(sector_encoding2,inplace=True)     # replace values 
    
    
    trace1 = go.Scatter3d(x = dataset["PCA_1"],   # Exponential PCA
                          y = dataset["PCA_2"],
                          z = dataset["PCA_3"], # np.exp(dataset["y_return"])*10,  #)
                          mode = "markers+text",
                          hovertext =  dataset[["Ticker","gsector","Date"]].values.tolist(),   #dataset["Ticker"].tolist(),                          hoverinfo="text",
                          # "X1_REVGH","X2_EPS"
                          
                          
#                           hovertemplate =
#                           '<i>F1</i>: $%{y:.2f}'+
#                           '<br><b>F2</b>: %{x}<br>'+
#                           '<b>%{text}</b>',
                          
#                           text = 'Custom text {}'.format(dataset["X1_REVGH"].values),#.tolist()),                        
                          
                          
                          #'<i>Feature 1.</i>: %{dataset["X1_REVGH"]:.2f}'+
                          #'<i>Feature 2.</i>' : '{dataset["X2_EPS"].values.tolist()}',
                          #'<b>%{text}</b>',
                          #hovertext = 
                          
                          marker = dict(size= abs((np.exp(df["y_return"])**4)*2), # scaled to increse std #np.exp(dataset["y_return"]+1.5)*2,#*20   # SCALE FIX
                                        sizemin = 1.5,  # min size
                                        color= dataset["Cluster"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        #name = list(df["Cluster"].unique()),
                                        colorbar = {"thickness": 10, "len": 0.5, "x": 0.8, "y": 0.6, },
                                        symbol = ['cross', 'diamond', 'square', 'square-open', # number markers == sectors
                                                  'square', 'diamond-open', 'cross', 'x', 'square','square', 'circle-open'],
                                        showscale = False, 
                                        
                                       #text = df["Ticker"].unique().tolist()
                                       
                                       ),  
                         
#                          hover_name = dataset["Ticker"], 
#                          hover_data = dataset.iloc[:, 4:24]
                         
                         
                         )
    conn.close() # Close DB

    data = [ trace1 ]
    
    
    layout = go.Layout( 
             title = "3D PCA Scatter Plot",
             # UX
             height = 1000,  #1500,
             width = 1000,  #1500,
             scene = dict( # UI
                          aspectmode = "cube",  
                          hovermode = "closest",
                          dragmode = "orbit", #  "orbit" | "turntable" | "zoom" | "pan" | False
                          xaxis = {"title": "PCA 1", },
                          yaxis = {"title": "PCA 2", },
                          zaxis = {"title": "PCA 3", },  )   ) #{"title": "Returns", },  )   ) 

#     layout = go.Layout( 
#               "xaxis": {
#               "rangeselector": {"title":"PCA 2",
#                                 "buttons": [  {"count": 6, "label": "6M", "step": "month",
#                                               "stepmode": "backward"},
#                                               {"count": 1, "label": "1Y", "step": "year",
#                                               "stepmode": "backward"},
#                                               {"count": 1, "label": "YTD", "step": "year",
#                                               "stepmode": "todate"},
#                                               {"label": "5Y", "step": "all",
#                                               "stepmode": "backward"}]                   } },
#               "yaxis": {"title":"PCA 2"},
#               "zaxis": {"title":"PCA 3"},  )
    
    fig =( {"data":data,
            "layout":layout})
    return fig , "Date Range {} - {}.".format(str(rangeD[0]),str(rangeD[1]))

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jan/2020 08:58:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2020 08:58:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2020 08:58:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2020 08:58:33] "GET /_favicon.ico?v=1.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2020 08:58:39] "POST /_dash-update-component HTTP/1.1" 200 -


In [71]:
df.head(1)

,YQuarter,Ticker,gsector,Date,X1_REVGH,X2_EPS,X3_ROA,X4_ROE,X5_PE,X6_PS,...,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12
0,1999Q3,A,Sector_35,1999-07-31 00:00:00,0.000267,0.251872,0.791643,0.974791,0.515213,0.000012,...,-0.08238,0.037696,-0.001543,-0.001425,-0.000367,0.0001,0.000303,0.000051,-0.000362,-0.000199


========================================== END ==============================================

In [ ]:
""" ATRTIBUTES TO LOOK FOR """
mode
surfacecolor
text 
textposition
hovertext
meta

AXIS
bordercolor
borderwidth
